In [1]:
import sympy as sp
import pandas as pd


In [2]:
# Alpha values to try:
# (we're assuming alpha=1 would cause divergence, not always the case though)

a_highestorder = 0
a_lowestorder = -10
alphas = []

for i in range(a_highestorder, a_lowestorder, -1):
    alphas.append(10**i)

alphas

[1, 0.1, 0.01, 0.001, 0.0001, 1e-05, 1e-06, 1e-07, 1e-08, 1e-09]

In [3]:
# The Data

x0 = [2, 5, 9]
x1 = [31, 56, 72]
ya = [240, 430, 562]


In [4]:
# Guessing initial weights and bias

w = [1, 2]
b = 3


In [5]:
# Create symbols need

w_s = sp.symbols('w0:2')
b_s = sp.symbols('b')


In [6]:
# Create cost symbolically

cost = 0
for i in range(0, len(ya)):
    diff = w_s[0]*x0[i] + w_s[1]*x1[i] + b_s - ya[i]
    cost += diff**2
cost = cost / (2*3)


In [7]:
# Figure out initial cost
w_b_vals = [ (w_s[0], w[0]), (w_s[1], w[1]), (b_s, b)]
round(cost.subs(w_b_vals))


48478

In [8]:
# Work out gradient of cost

grad_cost = []
for i in range(0, len(w)):
    dC_dwi = sp.diff(cost, w_s[i])
    grad_cost.append(dC_dwi)

dC_db = sp.diff(cost, b_s)
grad_cost.append(dC_db)


In [9]:
# The best alpha in this trial and error approach, will be the first one that DOES NOT diverge:


# Try out each alpha:
for j in range(0, len(alphas)):
    
    
    # Reset weights/bias
    w = [1, 2]
    b = 3
    costs_alpha = []
    isdivergent = False
    
    
    # Perform 1000 laps of training
    for i in range(0, 1000):

        # Pair the weights/bias, with their actual values
        w_b_vals = [ (w_s[0], w[0]), (w_s[1], w[1]), (b_s, b)]
        
        # Calculate cost
        cost_n = cost.subs(w_b_vals)
        costs_alpha.append(cost_n)
        
        # Check if cost is diverging
        if costs_alpha[i] > costs_alpha[i-1]:
            isdivergent = True
            print("alpha {} is causing divergence, moving onto next value".format(alphas[j]))
            break

        # Substitute values into all partial derivatives
        grad_cost_n = []
        for i in range(0, len(grad_cost)):
            deriv_n = grad_cost[i].subs(w_b_vals)
            grad_cost_n.append(deriv_n)

        # Update/change weights and bias
        w[0] = w[0] + alphas[j]*-grad_cost_n[0]
        w[1] = w[1] + alphas[j]*-grad_cost_n[1]
        b = b + alphas[j] * -grad_cost_n[2]
    
    # If no break occured, then we've found the largest alpha that did not diverge, thus it is the best
    if isdivergent == False:
        print("This alpha is the best: {}".format(alphas[j]))
        break


alpha 1 is causing divergence, moving onto next value
alpha 0.1 is causing divergence, moving onto next value
alpha 0.01 is causing divergence, moving onto next value
alpha 0.001 is causing divergence, moving onto next value
This alpha is the best: 0.0001


Note - Optimizing trial and error search:

To help reduce how long it takes to find a decent value for alpha, we calculate the cost during each lap of training, for each alpha value, and if the cost from a particular lap, goes up compared to the previous lap, we immediately know that for all subsequent laps (jumps), cost will only continue to go up (divergence will continue), so we know that this alpha value will cause divergence, and thus we'll stop training with it, and instead move onto the next alpha value.

This means we only need to do two laps of training for each diverging alpha value, then 1000 laps for the first non-diverging alpha value (compared to 1000 laps for EVERY alpha value, and then just compare cost for each, and see which was lowest).

HOWEVER, this optimization to trial and error finding for a decent alpha value, DOES NOT WORK, for NON-LINEAR regression models (as the graphs of cost in linear regression models, are always either parabolic, or the parabolic equivalent for higher dimensions, which have simple geometry, so that one jump of divergence, means all future jumps will also be divergent. Non-linear regression models DO NOT necessarily follow this behaviour)
